# Create a Dashboard Using Coronavirus Dataset

This project shows how to make a dashboard in Python using coronavirus dataset obtained from [EU Open Data Portal - COVID-19 Coronavirus Data ](https://data.europa.eu/euodp/en/data/dataset/covid-19-coronavirus-data).

Dash apps are composed of 2 parts: layout and interactivity. 
Layout can be set by html components.
Interactivity can be achieved using callbacks.


**Sources:**

1. https://dash.plotly.com/dash-core-components/tabs
2. https://medium.com/swlh/dashboards-in-python-for-beginners-and-everyone-else-using-dash-f0a045a86644
3. https://medium.com/a-r-g-o/using-plotlys-dash-to-deliver-public-sector-decision-support-dashboards-ac863fa829fb
4. https://www.w3schools.com/bootstrap/bootstrap_grid_system.asp

### 1. Import Necessary Libraries

In [1]:
import pandas as pd
import dash
import jupyterlab_dash

#These libraries are for the layout (visual components of the application) :
import dash_core_components as dcc #describe higher-level components that are interactive and are generated with 
                                   #JavaScript, HTML, and CSS through the React.js library.
import dash_html_components as html
from dash.dependencies import Input, Output # needed for callbacks
import dash_table # needed to make a table in dashboard
import plotly.express as px # needed to make map

### 2. Load the Data

In [2]:
# load data
df = pd.read_csv('./coronavirus_dataset.csv')

### 3. Clean Data

In [3]:
# clean data
df = df.rename(columns={'dateRep':'Date','countriesAndTerritories':'Country',
                        'countryterritoryCode':'Code','popData2018':'Population',
                        'cases':'Cases', 'deaths':'Deaths'})
# change date time
df.Date = pd.to_datetime(df.Date, format='%d/%m/%Y')

In [4]:
# make a list of all countries
country = df['Country'].unique()

In [5]:
# create a table for Data Table tab in dashboard
df_tab = df.groupby('Country')[['Cases','Deaths']].sum().reset_index()
df_tab.sort_values(by='Cases', ascending=False, inplace=True)

In [6]:
# create table for map
df_map = df.groupby('Country').agg({'Cases':'sum', 'Code':'max'}).reset_index()

### 3. Build Dashboard

In [7]:
# build app viewer
viewer = jupyterlab_dash.AppViewer() 

# build app
# set the fonts in app with external stylesheets(Bootstrap Grid System is used)
external_stylesheets = ['https://cdn.rawgit.com/plotly/dash-app-stylesheets/2d266c578d2a6e8850ebce48fdb52759b2aef506/stylesheet-oil-and-gas.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# create a buble map showing cases by country
fig = px.scatter_geo(df_map, locations="Code",
                     hover_name="Country", size="Cases",
                     projection="natural earth")
fig.update_geos(showcountries=True) # show country borders
fig.update_layout(title_text = 'Covid-19 Cases in the World')

# add two tabs in layout
# id and values for dcc.Tabs will be input for the callback
# id value 'tabs-content' will be output for the callback
app.layout = html.Div([
    dcc.Tabs(
        id = "tabs", value= 'tab_p', children =[
            dcc.Tab(label='Data Table', value = 'tab-1'
            ),
            dcc.Tab(label='Analysis Dashboard', value = 'tab-2'
            ),
        ]),
    html.Div(id='tabs-content')
])
        
#callback to control the tab content
@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def display_content(tab):
    if tab == 'tab-1':
        return html.Div([ 
            dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i } for i in df_tab.columns],
    style_cell={'textAlign': 'left',
                'border': '1px solid grey'},
    style_header={
        'border': '1px solid black',
        'backgroundColor': 'white',
        'fontWeight': 'bold'
    },
    data=df_tab.to_dict('records')
             )
        ])    
    elif tab == 'tab-2':
        return html.Div([
            html.Div([
            dcc.Dropdown(id='dropdown',
            options=[{
                'label':i,
                'value':i 
            } for i in country],
            value=df.Country[0]),    
            ],
            style={'width': '25%',
               'display': 'inline-block'}),
            html.Div([
            dcc.Graph(id='case-by-country')],
             className = 'twelve columns'), # bootstrap grid system for layout
            html.Div([
            dcc.Graph(id='map', figure = fig)],
             className = 'twelve columns')  # bootstrap grid system for layout
        ])   

#callback to control the graph
@app.callback(Output('case-by-country', 'figure'), [Input('dropdown', 'value')])
def update_graph(country):
    d_fig = df.loc[df['Country'] == country]
    return px.line(d_fig, x='Date', y='Cases', 
                   title='Number of Cases for each day in ' + str(country))

viewer.show(app)